<a href="https://colab.research.google.com/github/FRED984/text_generator_with_RNN/blob/main/Text_generator_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np
import keras

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
train_data[0]


[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 70s 108ms/step - loss: 0.4661 - acc: 0.7659 - val_loss: 0.3217 - val_acc: 0.8702
Epoch 2/10
625/625 [==============================] - 69s 111ms/step - loss: 0.2744 - acc: 0.8936 - val_loss: 0.2983 - val_acc: 0.8828
Epoch 3/10
625/625 [==============================] - 78s 124ms/step - loss: 0.2101 - acc: 0.9231 - val_loss: 0.2894 - val_acc: 0.8804
Epoch 4/10
625/625 [==============================] - 87s 139ms/step - loss: 0.1671 - acc: 0.9406 - val_loss: 0.3366 - val_acc: 0.8854
Epoch 5/10
625/625 [==============================] - 78s 125ms/step - loss: 0.1388 - acc: 0.9519 - val_loss: 0.3173 - val_acc: 0.8824
Epoch 6/10
625/625 [==============================] - 81s 130ms/step - loss: 0.1147 - acc: 0.9623 - val_loss: 0.3126 - val_acc: 0.8792
Epoch 7/10
625/625 [==============================] - 85s 136ms/step - loss: 0.0960 - acc: 0.9696 - val_loss: 0.3602 - val_acc: 0.8766
Epoch 8/10
625/625 [==============================] - 7

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)


782/782 [==============================] - 20s 25ms/step - loss: 0.4705 - acc: 0.8498
[0.4704665541648865, 0.8497599959373474]


In [ ]:
model.save("RNN.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
new_model = tf.keras.models.load_model("RNN.h5")

Making predictions

In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text +=reverse_word_index[num] + " "


  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was awesome! I really love it and would watch it again because it was amazingly great "
predict(positive_review)

negative_review = "that movie sucked. I hated it and wouldn't watch it again . Was one of the worst thing I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 495ms/step
[0.86067504]
1/1 [==============================] - 0s 40ms/step
[0.25067896]


**Text Generation**

Getting Shakespeare dataset

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

Loading my own dataset into google collab

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [ ]:
#Read , then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding ='utf-8')
# length of text is the number of character in it
print ( 'Length of text : {} character'.format(len(text)))

Length of text : 1115394 character


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Encoding

Since the text isn't encoded yet we need to encode it ourself.We are going to encode  each unique character as a different character

In [ ]:
vocab = sorted(set(text))

#Creating a mappping froom  unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
#let's look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
seq_length = 100 #length of  sequence for a training example
example_per_epoch = len(text)//(seq_length+1)

#Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characteres into batches of desired length

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
F

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
a

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

#Buffer size to shuffle the dataset

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Bulding the model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True,
                           recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in  data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for a prediction on our first batch of training data
  print(example_batch_predictions.shape, "(batch_size, sequence_length, vocab_size)") #print out the output shape

(64, 1, 65) (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 0.00260171  0.00094134  0.00285054 ...  0.00448469 -0.00014597
   -0.0016118 ]]

 [[-0.00022803  0.00204921 -0.0009244  ...  0.00243076  0.00334891
    0.00226893]]

 [[-0.00341004  0.00928213  0.00568374 ...  0.00175959  0.00089707
    0.00137601]]

 ...

 [[ 0.00148342  0.0058857   0.00152939 ...  0.00156662  0.00034582
    0.00223947]]

 [[ 0.0013831   0.00226525 -0.0044385  ...  0.00181961 -0.00015053
    0.0028406 ]]

 [[-0.00637813  0.00437854 -0.00040373 ... -0.00218587 -0.0010261
   -0.00152669]]], shape=(64, 1, 65), dtype=float32)


In [ ]:
# let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next

1
tf.Tensor(
[[ 2.6017108e-03  9.4133965e-04  2.8505402e-03 -9.3627826e-04
  -5.6448514e-03  4.1216577e-04  1.3217230e-03  3.8447573e-03
  -3.3815112e-04 -2.7553877e-03 -5.6547774e-03  2.1169763e-03
  -6.8874061e-03 -3.5049296e-03  3.3254190e-03 -1.1332575e-03
   1.0287001e-03 -1.3918793e-03  4.5754132e-04  2.9779074e-03
   2.3151322e-03  8.0871470e-03 -5.8574718e-04 -9.4085489e-04
   2.9233717e-03 -2.5051706e-03  3.0229539e-03 -4.2441970e-04
  -2.5586607e-03 -4.6600197e-03  2.6047085e-03 -1.7506464e-03
  -7.2543335e-04  4.0303599e-03 -1.8295294e-03 -1.6390284e-03
  -1.9447693e-03 -1.6999420e-03 -7.0043029e-03  3.3243257e-04
  -1.5047812e-03 -4.5793154e-03  1.1377543e-03 -2.3780838e-03
  -4.7986638e-03  5.8649420e-03 -5.9739044e-03  1.1070175e-03
  -3.2567461e-03  1.3540724e-03 -1.3064244e-05  1.3104961e-03
   2.1081115e-03  3.0668167e-04  2.0315382e-04 -2.8882492e-03
  -2.3048026e-03  6.6770595e-03  5.7895547e-03 -5.2187522e-04
   1.9725007e-03  4.7238925e-03  4.4846879e-03 -1.4596607

In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probability of each character occuring next

65
tf.Tensor(
[ 2.6017108e-03  9.4133965e-04  2.8505402e-03 -9.3627826e-04
 -5.6448514e-03  4.1216577e-04  1.3217230e-03  3.8447573e-03
 -3.3815112e-04 -2.7553877e-03 -5.6547774e-03  2.1169763e-03
 -6.8874061e-03 -3.5049296e-03  3.3254190e-03 -1.1332575e-03
  1.0287001e-03 -1.3918793e-03  4.5754132e-04  2.9779074e-03
  2.3151322e-03  8.0871470e-03 -5.8574718e-04 -9.4085489e-04
  2.9233717e-03 -2.5051706e-03  3.0229539e-03 -4.2441970e-04
 -2.5586607e-03 -4.6600197e-03  2.6047085e-03 -1.7506464e-03
 -7.2543335e-04  4.0303599e-03 -1.8295294e-03 -1.6390284e-03
 -1.9447693e-03 -1.6999420e-03 -7.0043029e-03  3.3243257e-04
 -1.5047812e-03 -4.5793154e-03  1.1377543e-03 -2.3780838e-03
 -4.7986638e-03  5.8649420e-03 -5.9739044e-03  1.1070175e-03
 -3.2567461e-03  1.3540724e-03 -1.3064244e-05  1.3104961e-03
  2.1081115e-03  3.0668167e-04  2.0315382e-04 -2.8882492e-03
 -2.3048026e-03  6.6770595e-03  5.7895547e-03 -5.2187522e-04
  1.9725007e-03  4.7238925e-03  4.4846879e-03 -1.4596607e-04
 -1.611797

In [ ]:
#If we want to determine the predicted character we need  to sample the output distribution(pick a value based on probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert  all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1,-1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

';'

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits,from_logits=True)

Compiling the model

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoint will be saved
checkpoint_dir = './training_checkpoints'
#Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback= tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weight_only=True,
    save_freq=5
)

Training the model

In [ ]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "<ipython-input-66-eaf5a3a84a38>", line 2, in loss  *
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits,from_logits=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 2454, in sparse_categorical_crossentropy
        return backend.sparse_categorical_crossentropy(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/backend.py", line 5775, in sparse_categorical_crossentropy
        res = tf.nn.sparse_softmax_cross_entropy_with_logits(

    ValueError: `labels.shape` must equal `logits.shape` except for the last dimension. Received: labels.shape=(64, 100) and logits.shape=(64, 1, 65)


Loading the model

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load  any  checkpoint we want by specifying the exact file to load

In [ ]:
Checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt"+ str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

**Generating Text**


In [ ]:
def generate_text(model, start_string):
  #Evaluation step (generating text using the learned model)

  #Number of characters to generate
  num_generate = 800

  #Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  #Empty string to store our results
  text_generated = []

  #Low temperature results in more predictable text.
  #Higher temperature results in more surprising text.
  #Experiment to find the best setting .
  temperature = 1.0

  #Here batch size ==1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    #We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expands_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_sting + ''.join(text_generated))



In [ ]:
inp = input("Type a starting string ")
print(generate_text(model, inp))